# INSTRUCTIONS

Run each cell in order. Put or extract data into relevant folders as defined in section 2. Make remaining folders in drive.  **Cells denoted with * may require extra action.**


```
Folder Structure Suggested for Section 2:
drive/
    train_data/ <- input midis go here
    train_output/
        graphs/ <- train/val accuracy plots go here
        intermed/ <- intermediate weights, preprocessing go here
        stats/ (currently not being used)
        midi/ <- output midis go here
```

# 1 Imports

In [1]:
# Import Data Manip, Debug
import glob
import pickle
import numpy as np
import pandas as pd
import pdb
import matplotlib.pyplot as plt

# Import Music21
!pip install music21
from music21 import converter, instrument, note, chord, stream

# Import Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import CuDNNLSTM
from keras.layers import CuDNNGRU
from keras.layers import Activation
from keras.layers import Bidirectional
from keras.layers import RepeatVector
from keras.layers import Flatten
from keras.regularizers import L1L2
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.callbacks import History

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


# 2 Mount and Set Directories*

In [2]:
DATA_FOLDER = 'drive/train_data/edm_all'
GRAPHS_FOLDER = 'drive/train_output/graphs'
MIDI_OUTPUT_FOLDER = 'drive/train_output/midi'
INTERMED_FOLDER = 'drive/train_output/intermed'
STATS_FOLDER = 'drive/train_output/stats'

# data specific
EDM_CORPUS = '/edm_'

# SET THIS TO ONE OF ABOVE (ACTION)
CORPUS = EDM_CORPUS

# 3 Set Training Parameters*

In [3]:
# SET PARAMETERS (ACTION)
RESTS = False
# SPECIFY PARAMETERS TO TEST AS LIST
DROPOUTS = [1, 0.5, 0.3]
MODEL_SIZES = [64, 256, 512]
EPOCHS = 200

# 4 Preprocess Data (MIDI Data Into Notes Corpus and Duration Corpus)

In [4]:
# Test

notes_corpus = []
durations_corpus = []

for file in glob.glob(DATA_FOLDER + "/*.mid"):
    try:
      print("Extracting MIDI File: ", file)
      midi_stream = converter.parse(file)

      notes = None

      partition = instrument.partitionByInstrument(midi_stream)

      if not RESTS:
        # No rests
        if partition: 
            notes = partition.parts[0].recurse()
        else: 
            notes = midi_stream.flat.notes
      else:
        # With rests
        if partition: 
            notes = partition.parts[0].recurse()
        else: 
            notes = midi_stream.notesAndRests
          
      in_song_notes = []
      in_song_durations = []
      for element in notes:
          in_song_durations.append(element.duration.quarterLength)
          if isinstance(element, note.Note):
              in_song_notes.append(str(element.pitch))
          elif RESTS and isinstance(element, note.Rest):
              in_song_notes.append("R")
          elif isinstance(element, chord.Chord):
              in_song_notes.append(element.root().nameWithOctave)
#               in_song.append('.'.join(str(n) for n in element.normalOrder))
    except:
      pass
    
    notes_corpus.append(in_song_notes)
    durations_corpus.append(in_song_durations)
            
# Write
with open(INTERMED_FOLDER + CORPUS + 'notes_corpus', 'wb+') as filepath:
    pickle.dump(notes_corpus, filepath)
    
with open(INTERMED_FOLDER + CORPUS + 'durations_corpus', 'wb+') as filepath:
    pickle.dump(durations_corpus, filepath)

Extracting MIDI File:  drive/train_data/edm_all\Afrojack _ David Guetta - Another Life  (midi by Carlo Prato) (www.cprato.com).mid
Extracting MIDI File:  drive/train_data/edm_all\Alan Walker - Alone  (midi by Carlo Prato) (www.cprato.com).mid
Extracting MIDI File:  drive/train_data/edm_all\Alan Walker - Faded (Original Mix) (midi by Carlo Prato) (www.cprato.com).mid
Extracting MIDI File:  drive/train_data/edm_all\Alan Walker - Sing Me To Sleep  (midi by Carlo Prato) (www.cprato.com).mid
Extracting MIDI File:  drive/train_data/edm_all\Alan Walker - Tired  (midi by Carlo Prato) (www.cprato.com).mid
Extracting MIDI File:  drive/train_data/edm_all\Alesso ft. Matthew Koma - Years (Original Mix) (midi by Carlo Prato) (www.cprato.com).mid
Extracting MIDI File:  drive/train_data/edm_all\Avicii - Dear Boy.mid
Extracting MIDI File:  drive/train_data/edm_all\Avicii - Lonely Together.mid
Extracting MIDI File:  drive/train_data/edm_all\Avicii - Without You.mid
Extracting MIDI File:  drive/train_dat

# 5 Preprocess Corpus Into Train Data

## 5.1 Notes Corpus

In [5]:
# load notes corpus
with open(INTERMED_FOLDER + CORPUS + 'notes_corpus', 'rb') as filepath:
    notes_corpus = pickle.load(filepath)
    
# If doing learning for one song at a time only
flattened_notes_corpus = []
for song_notes in notes_corpus:
    flattened_notes_corpus += song_notes

vocab_size = len(set(flattened_notes_corpus))
print(vocab_size)

# Produce input output sequences
window_size = 60
note_sequence_input = []
next_note_output = []

notes = sorted(set(flattened_notes_corpus))
note2int = dict((note, num) for num, note in enumerate(notes))

for i in range(0, len(notes_corpus)):
    for j in range(0, len(notes_corpus[i]) - window_size):
        current_sequence = [note2int[note] for note in notes_corpus[i][j:window_size+j]]
        next_note = note2int[notes_corpus[i][window_size+j]]
        note_sequence_input.append(current_sequence)
        next_note_output.append(next_note)

68


In [6]:
# check train and label shapes
training_data = np.reshape(note_sequence_input, (len(note_sequence_input), window_size , 1))
training_data = training_data / float(vocab_size)
print('Train shape: ' + str(training_data.shape))
training_label = np_utils.to_categorical(next_note_output)
print('Label shape: ' + str(training_label.shape))

Train shape: (6258, 60, 1)
Label shape: (6258, 68)


## 5.2 Durations Corpus

In [7]:
# load notes corpus
with open(INTERMED_FOLDER + CORPUS + 'durations_corpus', 'rb') as filepath:
    duration_corpus = pickle.load(filepath)

# Learn on one song at a time
flattened_duration_corpus = []
for song_durations in duration_corpus:
    flattened_duration_corpus += song_durations
    
import collections
counter = collections.Counter(flattened_duration_corpus)
print(counter)


duration_vocab_size = len(set(flattened_duration_corpus))
print(duration_vocab_size)

# Produce input output sequences
duration_window_size = 30
duration_sequence_input = []
next_duration_output = []

durations = sorted(set(flattened_duration_corpus))
duration2int = dict((duration, num) for num, duration in enumerate(durations))

print(duration2int)
# Write
with open(INTERMED_FOLDER + "/edm_duration_counter", 'wb+') as filepath:
    pickle.dump(counter, filepath)

for i in range(0, len(duration_corpus)):
    for j in range(0, len(duration_corpus[i]) - duration_window_size):
        current_duration_sequence = [duration2int[note] for note in duration_corpus[i][j:duration_window_size+j]]
        next_duration = duration2int[duration_corpus[i][duration_window_size+j]]
        duration_sequence_input.append(current_duration_sequence)
        next_duration_output.append(next_duration)

Counter({0.25: 5380, 0.5: 4464, Fraction(1, 3): 1009, 0.0: 998, 1.0: 741, Fraction(1, 6): 729, 0.75: 485, 4.0: 166, 2.0: 130, Fraction(2, 3): 98, Fraction(1, 12): 86, Fraction(5, 12): 72, 1.5: 59, 3.0: 41, Fraction(7, 6): 32, 1.25: 31, 1.75: 30, 2.25: 25, 4.5: 22, 3.5: 13, 6.5: 13, 8.0: 11, 4.25: 9, 2.5: 9, Fraction(25, 6): 7, 5.0: 6, 7.0: 5, 8.5: 4, 32.0: 4, 6.0: 4, Fraction(5, 3): 3, 48.0: 2, 68.0: 2, 80.0: 2, 16.0: 2, 176.0: 2, 47.5: 1, 3.25: 1, Fraction(23, 3): 1, 5.75: 1, 14.5: 1, 9.0: 1, 200.5: 1, 36.25: 1, 64.25: 1, 84.25: 1, 72.5: 1, 72.25: 1, 61.0: 1, 27.5: 1, 161.0: 1, 128.25: 1, 18.0: 1, 39.5: 1, 32.5: 1, 36.0: 1, 94.5: 1, 5.25: 1, 4.75: 1, Fraction(553, 6): 1, Fraction(4, 3): 1})
61
{0.0: 0, Fraction(1, 12): 1, Fraction(1, 6): 2, 0.25: 3, Fraction(1, 3): 4, Fraction(5, 12): 5, 0.5: 6, Fraction(2, 3): 7, 0.75: 8, 1.0: 9, Fraction(7, 6): 10, 1.25: 11, Fraction(4, 3): 12, 1.5: 13, Fraction(5, 3): 14, 1.75: 15, 2.0: 16, 2.25: 17, 2.5: 18, 3.0: 19, 3.25: 20, 3.5: 21, 4.0: 22, Fr

In [8]:
# check train and label shapes
duration_training_data = np.reshape(duration_sequence_input, (len(duration_sequence_input), duration_window_size , 1))
duration_training_data = duration_training_data / float(duration_vocab_size)
print('Train shape: ' + str(duration_training_data.shape))
duration_training_label = np_utils.to_categorical(next_duration_output)
print('Label shape: ' + str(duration_training_label.shape))

Train shape: (13030, 30, 1)
Label shape: (13030, 61)


# 6 Train

## 6.1 Helper to Create Model

In [9]:
def create_model(network_input, n_vocab, model_size, dropout):
  model = Sequential()
  reg = L1L2(0, 0)
  model.add(LSTM(
      model_size,
      input_shape=(network_input.shape[1], network_input.shape[2]),
      return_sequences=True,
      dropout=dropout, recurrent_dropout=0.3
  ))
  model.add(CuDNNLSTM(model_size, return_sequences=True, kernel_regularizer=reg))
  model.add(Dropout(dropout))
  model.add(CuDNNLSTM(model_size, kernel_regularizer=reg))
  model.add(Dense(128))
  model.add(Dropout(dropout))
  model.add(Dense(n_vocab))
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_callback_list(model_size, dropout, model_type):
  filepath = INTERMED_FOLDER + '/%skarpathy-model-weights-%s-%s-%s.hdf5' % (CORPUS, model_type, model_size, dropout)
  model_checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
  )
  return [model_checkpoint], filepath

# acc history
def setup_plot(dropout, size):
  plt.title('Model Accuracy vs. Epoc with Dropout=%s Size=%s' % (dropout, size))
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  
def plot_history(history, model_type, dropout, size):
  plt.plot(history.history['acc'], label="%s train accuracy" % model_type)
  plt.plot(history.history['val_acc'], label="%s val accuracy" % model_type)

def save_plot(file_path):
  plt.legend()
  plt.savefig(file_path)
  plt.clf()
  
def predict_duration(model, WEIGHT_PATH):
  # Prediction
  model.load_weights(WEIGHT_PATH)
  starting_sequence = np.random.randint(219, size=duration_window_size)
  pattern_sequence = starting_sequence.tolist()
  prediction_output = []

  duration2note = dict((num, note) for num, note in enumerate(durations))
  print (duration2note)
  
  for i in range(400):
      prediction_input = np.reshape(pattern_sequence, (1, len(pattern_sequence), 1))
      prediction_input = prediction_input / float(duration_vocab_size)

      if i%3 == 0:   
          print('\r', 'Predicting.  Duration: ', i, end='')  
      if i%3 == 1:   
          print('\r', 'Predicting.. Duration: ', i, end='')
      if i%3 == 2:   
          print('\r', 'Predicting...Duration: ', i, end='')
      prediction = model.predict(prediction_input, verbose=0)

#       prediction_values = np.arange(len(prediction[0]))
#       prediction_prob = np.asarray(list(prediction[0])) / float(sum(prediction[0]))

      # Most probable note prediction
      index = np.argmax(prediction)
      note_instance = duration2note[index]
      prediction_output.append(note_instance)
#       index = np.random.choice(prediction_values, 1, p=prediction_prob)
#       note_instance = int2note[int(index[0])]
#       prediction_output.append(note_instance)

      pattern_sequence.append(index)
      pattern_sequence = pattern_sequence[1:len(pattern_sequence)]    

  prediction_output = prediction_output[300:len(prediction_output)]

  # Write
  with open(INTERMED_FOLDER + ("%sduration_prediction_output" % CORPUS), 'wb+') as filepath:
      pickle.dump(prediction_output, filepath)
  
  return prediction_output

def predict_note(model, WEIGHT_PATH):
  model.load_weights(WEIGHT_PATH)
  # Prediction
  starting_sequence = np.random.randint(219, size=window_size)
  pattern_sequence = starting_sequence.tolist()
  prediction_output = []

  int2note = dict((num, note) for num, note in enumerate(notes))
  print (int2note)

  for i in range(400):
      prediction_input = np.reshape(pattern_sequence, (1, len(pattern_sequence), 1))
      prediction_input = prediction_input / float(vocab_size)

      if i%3 == 0:   
          print('\r', 'Predicting.  Note: ', i, end='')  
      if i%3 == 1:   
          print('\r', 'Predicting.. Note: ', i, end='')
      if i%3 == 2:   
          print('\r', 'Predicting...Note: ', i, end='')
      prediction = model.predict(prediction_input, verbose=0)

#       prediction_values = np.arange(len(prediction[0]))
#       prediction_prob = np.asarray(list(prediction[0])) / float(sum(prediction[0]))
#       print(prediction_prob)

      # Most probable note prediction
      index = np.argmax(prediction)
      note_instance = int2note[index]
      prediction_output.append(note_instance)

      # Predict based on prob dist
#       index = np.random.choice(prediction_values, 1, p=prediction_prob)
  #     print(index[0])
  #     print(type(index[0]))
#       note_instance = int2note[int(index[0])]
#       prediction_output.append(note_instance)

      pattern_sequence.append(index)
      pattern_sequence = pattern_sequence[1:len(pattern_sequence)]    

  prediction_output = prediction_output[300:len(prediction_output)]
  # Write
  with open(INTERMED_FOLDER + ("%snotes_prediction_output" % CORPUS), 'wb+') as filepath:
      pickle.dump(prediction_output, filepath)
  return prediction_output

def output_midi(prediction_output, duration_prediction_output, dropout, model_size):
  offset = 0
  output_notes = []
  for pattern, duration in zip(prediction_output, duration_prediction_output):

      if ('.' in pattern) or pattern.isdigit():
          chord_array = pattern.split('.')
          chord_notes = []
          for note_instance in chord_array:
              note_object = note.Note(int(note_instance))
              note_object.duration.quarterLength = duration
              note_object.storedInstrument = instrument.Piano()
              chord_notes.append(note_object)
          chord_object = chord.Chord(chord_notes)
          chord_object.offset = offset
          output_notes.append(chord_object)
      elif 'R' == pattern:
          note_object = note.Rest()
          note_object.duration.quarterLength = duration
          note_object.offset = offset
          output_notes.append(note_object)
      else:
          note_object = note.Note(pattern)
          note_object.duration.quarterLength = duration
          note_object.offset = offset
          note_object.storedInstrument = instrument.Piano()
          output_notes.append(note_object)

      offset += 0.5

  midi_stream = stream.Stream(output_notes)
  midi_stream.write('midi', fp=MIDI_OUTPUT_FOLDER + CORPUS + '%s_%s.mid' % (dropout, model_size))
  print('Wrote midi...')

In [11]:
notes_histories = {}
duration_histories = {}
for dropout in DROPOUTS:
  for model_size in MODEL_SIZES:
    setup_plot(dropout, model_size)
    print('Running duration training on dropout:%s size:%s' % (dropout, model_size))
    duration_callbacks, duration_weight_path = create_callback_list('duration', dropout, model_size)
    duration_model = create_model(duration_training_data, duration_vocab_size, model_size, dropout)
    duration_histories[(dropout, model_size)] = duration_model.fit(duration_training_data, duration_training_label, epochs=EPOCHS, batch_size=256, callbacks=duration_callbacks, validation_split=0.2)
    plot_history(duration_histories[(dropout, model_size)], 'Durations', dropout, model_size)
    # output intermed duration
    duration_prediction = predict_duration(duration_model, duration_weight_path)
    
    print('\n\nRunning notes training on d:%s s:%s' % (dropout, model_size))
    notes_callbacks, note_weight_path = create_callback_list('notes', dropout, model_size)
    notes_model = create_model(training_data, vocab_size, model_size, dropout)
    notes_histories[(dropout, model_size)] = notes_model.fit(training_data, training_label, epochs=EPOCHS, batch_size=124, callbacks=notes_callbacks, validation_split=0.2)
    plot_history(notes_histories[(dropout, model_size)], 'Notes', dropout, model_size)
    save_plot(GRAPHS_FOLDER + EDM_CORPUS + 'dropout=%s_size=%s.jpg' % (dropout, model_size))
    # output intermed notes
    note_prediction = predict_note(notes_model, note_weight_path)
    
    # output final midi
    output_midi(note_prediction, duration_prediction, dropout, model_size)

Running duration training on dropout:1 size:64
Train on 10424 samples, validate on 2606 samples
Epoch 1/200
10424/10424 [==============================] - 5s 446us/step - loss: 2.4440 - acc: 0.3731 - val_loss: 1.9200 - val_acc: 0.3097
Epoch 2/200
10424/10424 [==============================] - 2s 213us/step - loss: 1.8243 - acc: 0.3804 - val_loss: 1.9032 - val_acc: 0.3097
Epoch 3/200
10424/10424 [==============================] - 2s 238us/step - loss: 1.8107 - acc: 0.3897 - val_loss: 1.9268 - val_acc: 0.3070
Epoch 4/200
10424/10424 [==============================] - 3s 252us/step - loss: 1.7117 - acc: 0.4591 - val_loss: 1.8235 - val_acc: 0.4094
Epoch 5/200
10424/10424 [==============================] - 2s 240us/step - loss: 1.6089 - acc: 0.4869 - val_loss: 1.8178 - val_acc: 0.4578
Epoch 6/200
10424/10424 [==============================] - 2s 226us/step - loss: 1.5517 - acc: 0.4958 - val_loss: 1.7998 - val_acc: 0.4612
Epoch 7/200
10424/10424 [==============================] - 2s 223us/st

Epoch 59/200
10424/10424 [==============================] - 2s 216us/step - loss: 1.1115 - acc: 0.5997 - val_loss: 2.5424 - val_acc: 0.3319
Epoch 60/200
10424/10424 [==============================] - 2s 213us/step - loss: 1.0938 - acc: 0.6106 - val_loss: 2.6814 - val_acc: 0.3070
Epoch 61/200
10424/10424 [==============================] - 2s 219us/step - loss: 1.0846 - acc: 0.6113 - val_loss: 2.6320 - val_acc: 0.3423
Epoch 62/200
10424/10424 [==============================] - 2s 211us/step - loss: 1.0770 - acc: 0.6177 - val_loss: 2.6168 - val_acc: 0.3277
Epoch 63/200
10424/10424 [==============================] - 2s 212us/step - loss: 1.0989 - acc: 0.6034 - val_loss: 2.6976 - val_acc: 0.3131
Epoch 64/200
10424/10424 [==============================] - 2s 210us/step - loss: 1.0408 - acc: 0.6333 - val_loss: 2.7291 - val_acc: 0.3335
Epoch 65/200
10424/10424 [==============================] - 2s 216us/step - loss: 1.0252 - acc: 0.6365 - val_loss: 2.8317 - val_acc: 0.3216
Epoch 66/200
10424/1

10424/10424 [==============================] - 2s 214us/step - loss: 0.4589 - acc: 0.8418 - val_loss: 4.9646 - val_acc: 0.3254
Epoch 118/200
10424/10424 [==============================] - 2s 213us/step - loss: 0.4557 - acc: 0.8408 - val_loss: 4.9691 - val_acc: 0.3338
Epoch 119/200
10424/10424 [==============================] - 2s 212us/step - loss: 0.4478 - acc: 0.8481 - val_loss: 4.9919 - val_acc: 0.3365
Epoch 120/200
10424/10424 [==============================] - 2s 223us/step - loss: 0.4449 - acc: 0.8498 - val_loss: 5.0259 - val_acc: 0.3327
Epoch 121/200
10424/10424 [==============================] - 2s 213us/step - loss: 0.4401 - acc: 0.8499 - val_loss: 4.9869 - val_acc: 0.3296
Epoch 122/200
10424/10424 [==============================] - 2s 215us/step - loss: 0.4381 - acc: 0.8489 - val_loss: 5.0035 - val_acc: 0.3496
Epoch 123/200
10424/10424 [==============================] - 2s 217us/step - loss: 0.6019 - acc: 0.7931 - val_loss: 4.9279 - val_acc: 0.3239
Epoch 124/200
10424/10424 [

10424/10424 [==============================] - 2s 216us/step - loss: 0.2328 - acc: 0.9200 - val_loss: 6.1264 - val_acc: 0.3503
Epoch 176/200
10424/10424 [==============================] - 2s 214us/step - loss: 0.2312 - acc: 0.9224 - val_loss: 6.1230 - val_acc: 0.3396
Epoch 177/200
10424/10424 [==============================] - 2s 215us/step - loss: 0.2274 - acc: 0.9220 - val_loss: 6.1192 - val_acc: 0.3538
Epoch 178/200
10424/10424 [==============================] - 2s 214us/step - loss: 0.2102 - acc: 0.9299 - val_loss: 6.1646 - val_acc: 0.3480
Epoch 179/200
10424/10424 [==============================] - 2s 217us/step - loss: 0.2033 - acc: 0.9330 - val_loss: 6.1585 - val_acc: 0.3404
Epoch 180/200
10424/10424 [==============================] - 2s 213us/step - loss: 0.2091 - acc: 0.9306 - val_loss: 6.2629 - val_acc: 0.3454
Epoch 181/200
10424/10424 [==============================] - 2s 215us/step - loss: 0.2228 - acc: 0.9217 - val_loss: 6.1867 - val_acc: 0.3411
Epoch 182/200
10424/10424 [

5006/5006 [==============================] - 4s 846us/step - loss: 2.0024 - acc: 0.3518 - val_loss: 6.0935 - val_acc: 0.0375
Epoch 29/200
5006/5006 [==============================] - 4s 857us/step - loss: 1.9567 - acc: 0.3546 - val_loss: 6.2267 - val_acc: 0.0431
Epoch 30/200
5006/5006 [==============================] - 4s 842us/step - loss: 1.9397 - acc: 0.3624 - val_loss: 6.3046 - val_acc: 0.0359
Epoch 31/200
5006/5006 [==============================] - 4s 848us/step - loss: 1.9038 - acc: 0.3592 - val_loss: 6.5171 - val_acc: 0.0288
Epoch 32/200
5006/5006 [==============================] - 4s 862us/step - loss: 1.9226 - acc: 0.3712 - val_loss: 6.4298 - val_acc: 0.0256
Epoch 33/200
5006/5006 [==============================] - 4s 852us/step - loss: 1.8618 - acc: 0.3714 - val_loss: 6.4685 - val_acc: 0.0463
Epoch 34/200
5006/5006 [==============================] - 4s 849us/step - loss: 1.8162 - acc: 0.3827 - val_loss: 6.4855 - val_acc: 0.0359
Epoch 35/200
5006/5006 [=======================

...

5006/5006 [==============================] - 8s 2ms/step - loss: 2.0085 - acc: 0.3885 - val_loss: 5.8396 - val_acc: 0.0176
Epoch 28/200
5006/5006 [==============================] - 8s 2ms/step - loss: 1.9588 - acc: 0.4149 - val_loss: 6.0069 - val_acc: 0.0128
Epoch 29/200
5006/5006 [==============================] - 8s 2ms/step - loss: 2.0120 - acc: 0.4035 - val_loss: 5.9402 - val_acc: 0.0136
Epoch 30/200
5006/5006 [==============================] - 8s 2ms/step - loss: 1.9287 - acc: 0.4243 - val_loss: 5.9388 - val_acc: 0.0128
Epoch 31/200
5006/5006 [==============================] - 8s 2ms/step - loss: 1.8396 - acc: 0.4495 - val_loss: 6.3048 - val_acc: 0.0184
Epoch 32/200
5006/5006 [==============================] - 8s 2ms/step - loss: 1.7716 - acc: 0.4692 - val_loss: 6.4741 - val_acc: 0.01441s - loss: 1.7686 - acc
Epoch 33/200
5006/5006 [==============================] - 8s 2ms/step - loss: 1.7652 - acc: 0.4898 - val_loss: 6.3112 - val_acc: 0.0240
Epoch 34/200
5006/5006 [==============

Epoch 87/200
5006/5006 [==============================] - 8s 2ms/step - loss: 1.1912 - acc: 0.7119 - val_loss: 9.7614 - val_acc: 0.0072
Epoch 88/200
5006/5006 [==============================] - 8s 2ms/step - loss: 1.1782 - acc: 0.7165 - val_loss: 9.4329 - val_acc: 0.0312
Epoch 89/200
5006/5006 [==============================] - 8s 2ms/step - loss: 1.1617 - acc: 0.7175 - val_loss: 9.7398 - val_acc: 0.0176
Epoch 90/200
5006/5006 [==============================] - 8s 2ms/step - loss: 1.1830 - acc: 0.7099 - val_loss: 9.7002 - val_acc: 0.0248s - loss: 1.1880
Epoch 91/200
5006/5006 [==============================] - 8s 2ms/step - loss: 1.1344 - acc: 0.7227 - val_loss: 9.6327 - val_acc: 0.0168
Epoch 92/200
5006/5006 [==============================] - 8s 2ms/step - loss: 1.1575 - acc: 0.7173 - val_loss: 9.5845 - val_acc: 0.0120
Epoch 93/200
5006/5006 [==============================] - 8s 2ms/step - loss: 1.1295 - acc: 0.7273 - val_loss: 9.8475 - val_acc: 0.0128
Epoch 94/200
5006/5006 [========

5006/5006 [==============================] - 8s 2ms/step - loss: 0.1027 - acc: 0.9694 - val_loss: 10.8369 - val_acc: 0.0184
Epoch 147/200
5006/5006 [==============================] - 8s 2ms/step - loss: 0.1163 - acc: 0.9698 - val_loss: 10.3100 - val_acc: 0.0359
Epoch 148/200
5006/5006 [==============================] - 8s 2ms/step - loss: 0.1009 - acc: 0.9714 - val_loss: 10.5681 - val_acc: 0.0216
Epoch 149/200
5006/5006 [==============================] - 8s 2ms/step - loss: 0.1056 - acc: 0.9714 - val_loss: 10.7211 - val_acc: 0.0200
Epoch 150/200
5006/5006 [==============================] - 8s 2ms/step - loss: 0.0928 - acc: 0.9772 - val_loss: 10.5422 - val_acc: 0.0176
Epoch 151/200
5006/5006 [==============================] - 8s 2ms/step - loss: 0.0897 - acc: 0.9768 - val_loss: 10.7480 - val_acc: 0.0152
Epoch 152/200
5006/5006 [==============================] - 8s 2ms/step - loss: 0.1052 - acc: 0.9724 - val_loss: 10.5527 - val_acc: 0.0176
Epoch 153/200
5006/5006 [=======================

 Predicting.  Note:  399Predicting.  Note:  153Wrote midi...


<Figure size 432x288 with 0 Axes>